# Import


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install plotly_express

In [ ]:
!pip install geopandas

     |████████████████████████████████| 1.0 MB 7.3 MB/s 
     |████████████████████████████████| 15.4 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 28.5 MB/s 


In [5]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'

cluster_coord.csv		way_and_road_segment.csv
roadLink_merge_roadDB.csv	wayid_label_mapped.csv
route_network_with_cluster.csv


In [3]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'
trips_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'

# get all way_id

In [4]:
way_segment_df = pd.read_csv(road_data_path+"/way_and_road_segment.csv")
way_segment_df['road_seg'] = way_segment_df['road_seg'].map(lambda x: eval(x))
way_segment_df

,way_id,seg_num,road_seg
0,23730732,0,"(35, 36)"
1,23747253,0,"(283, 2864)"
2,23747922,0,"(2772, 2773)"
3,23747922,1,"(2773, 2774)"
4,23747922,2,"(2774, 2775)"
...,...,...,...
10630,983757864,0,"(4719, 4720)"
10631,983757864,1,"(3299, 4720)"
10632,983757865,0,"(4496, 4719)"
10633,983823827,0,"(3069, 4721)"


# get way_id with label

In [8]:
way_mapped_df = pd.read_csv(road_data_path+"/wayid_label_mapped.csv")
way_mapped_df['road_seg'] = way_mapped_df['road_seg'].map(lambda x: eval(x))
way_df = way_mapped_df[way_mapped_df['label'] != -1]
way_df['a'] = way_df['road_seg'].apply(lambda x: x[0])
way_df['b'] = way_df['road_seg'].apply(lambda x: x[1])
way_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,way_id,seg_num,road_seg,label,a,b
0,23730732,0,"(35, 36)",2146,35,36
5,23749257,0,"(1233, 1234)",2047,1233,1234
6,23749257,1,"(715, 1234)",2047,715,1234
10,23750525,0,"(1235, 1236)",3021,1235,1236
11,23751227,0,"(254, 2776)",3102,254,2776
...,...,...,...,...,...,...
10625,983567976,0,"(4426, 7663)",3106,4426,7663
10626,983751249,0,"(3296, 4718)",2102,3296,4718
10630,983757864,0,"(4719, 4720)",2104,4719,4720
10631,983757864,1,"(3299, 4720)",2104,3299,4720


In [51]:
way_df[way_df.b == 1233]

,way_id,seg_num,road_seg,label,a,b


In [47]:
len(way_df)

4652

In [30]:
a_count

35      8
1337    7
42      6
1333    6
888     6
       ..
4461    1
363     1
1621    1
3672    1
4096    1
Name: a, Length: 3029, dtype: int64

In [29]:
a_count[35]

8

In [55]:
a_count = way_df['a'].value_counts()
b_count = way_df['b'].value_counts()
groups = []

def in_groups(way_id):
  for i in groups:
    for j in i:
      if way_id == j:
        return True
  return False

def next_way(way, seg):
  if way == 'b':
    next_way_df = way_df[way_df.b == seg].iloc[0]
    next_way_id = next_way_df.way_id
    a = next_way_df.a
    b = seg
    return next_way_id, a, b

  if way == 'a':
    next_way_df = way_df[way_df.a == seg].iloc[0]
    next_way_id = next_way_df.way_id
    b = next_way_df.b
    a = seg
    return next_way_id, a, b

def get_count(count_series, seg):
  try:
    count = count_series[seg]
  except:
    count = 0
  return count

for way_id in way_df.way_id.to_list():

  if in_groups(way_id):
    continue
  else:
    way = way_df[way_df.way_id == way_id].iloc[0]
    a = way.a
    b = way.b

  # a route
  group_a = []
  while get_count(b_count, a) == 1:
    next_way_id, a_next, b_next = next_way('b', a)
    group_a.append(next_way_id)
    a = a_next

  # b route
  group_b = []
  while get_count(a_count, b) == 1:
    next_way_id, a_next, b_next = next_way('a', b)
    group_b.append(next_way_id)
    b = b_next

  group_a.reverse()
  group_a.append(way_id)
  group = group_a + group_b
  groups.append(group)

count = 0
for i in groups:
  count += len(i)
print(count)

3710


In [57]:
len(groups)

1400

In [58]:
set_g = set()
for group in groups:
  for g in group:
    set_g.add(g)

In [59]:
len(set_g)

1915